In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [11]:
%matplotlib notebook

In [12]:
L = 1
T = 10000
dT = 2
Nx = 500
Ny = 500
a = 0.0000001

In [22]:
x = np.linspace(0, L, Nx+1) # mesh points in space
y = np.linspace(0, L, Ny+1) 
dx = x[1] - x[0]
dy = y[1] - y[0]
t = np.arange(0, T, dT)    # mesh points in time
Nt = len(t)
Fx = a*dT/dx**2
Fy = a*dT/dy**2
u   = np.zeros((Nx+1, Ny+1)) # unknown u at new time level
u_1   = np.zeros((Nx+1, Ny+1))
dux = np.zeros(Nx+1)
duy = np.zeros(Ny+1)
du = np.zeros((Nx+1, Ny+1))
du = np.zeros((Nx+1, Ny+1))

# u_1[250, 100] = 1
# u_1[150, 250] = 0.5
# u_1[450, 300] = 1.5

In [23]:
def odefunc(u, t):
    dux = np.zeros((Nx+1, Ny+1))
    duy = np.zeros((Nx+1, Ny+1))
    du = np.zeros((Nx+1, Ny+1))

    for n in range(0, Nt):
     # Compute u at inner mesh points
        for i in range(1, Nx):
            for j in range(1, Ny):
                dux[i, j] = Fx*(u_1[i-1, j] - 2*u_1[i, j] + u_1[i+1, j]) 
                dux[0, j]=Fx*(u_1[Nx, j] - 2*u_1[0, j] + u_1[1, j]) 
                dux[Nx, j]=Fx*(u_1[Nx-1, j] - 2*u_1[Nx, j] + u_1[0, j]) 
            
                duy[i, j] = Fy*(u_1[i, j-1] - 2*u_1[i, j] + u_1[i, j+1])
                duy[i, 0]=Fy*(u_1[i, Ny] - 2*u_1[i, 0] + u_1[i, 1])
                duy[i, Ny]=Fy*(u_1[i, Ny-1] - 2*u_1[i, Ny] + u_1[i, 0])
                
                du = dux + duy
            
        for i in range(0, Nx+1):
            for j in range(0, Ny+1):
                u_1[i, j] = u_1[i,j] + du[i,j]


    return du

In [24]:
sol = odeint(odefunc,u_1,Nt)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
